In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from pathlib import Path
import xarray as xr

from opera_tropo import checks

import logging
logger = logging.getLogger(__name__)

In [3]:
import sys
# Method 1: Basic setup (simplest)
def setup_basic_logging():
    """Basic logging setup for Jupyter notebook"""
    logging.basicConfig(
        level=logging.INFO,
        format='%(levelname)s - %(name)s - %(message)s',
        handlers=[logging.StreamHandler(sys.stdout)]
    )

In [4]:
setup_basic_logging()

In [5]:
work_dir = Path('download_test').absolute()
filename = 'ECMWF_TROP_202409100600_202409100600_1.nc'

In [6]:
# Print the valid range
for id, key in checks.VALID_RANGE.items():
    print(f'{id} - {key}')

t - (140.0, 360.0)
q - (0.0, 0.3)
z - (-5000.0, 70000.0)
lnsp - (10.2, 11.75)


In [7]:
ds = xr.open_dataset(work_dir / filename, chunks={'level':-1})
#ds = ds.chunk({, 'longitude':256, 'latitude':256})

In [8]:
from dask.distributed import Client

client = Client(
    #n_workers=4,
    #threads_per_worker=2,
    #memory_limit=f"{30}GB",
    )
client.dashboard_link

INFO - distributed.http.proxy - To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO - distributed.scheduler - State start
INFO - distributed.diskutils - Found stale lock file and directory '/tmp/dask-scratch-space-75956/scheduler-tfnfu563', purging
INFO - distributed.scheduler -   Scheduler at:     tcp://127.0.0.1:34096
INFO - distributed.scheduler -   dashboard at:  http://127.0.0.1:8787/status
INFO - distributed.scheduler - Registering Worker plugin shuffle
INFO - distributed.nanny -         Start Nanny at: 'tcp://127.0.0.1:44627'
INFO - distributed.nanny -         Start Nanny at: 'tcp://127.0.0.1:35963'
INFO - distributed.nanny -         Start Nanny at: 'tcp://127.0.0.1:39225'
INFO - distributed.nanny -         Start Nanny at: 'tcp://127.0.0.1:39659'
INFO - distributed.nanny -         Start Nanny at: 'tcp://127.0.0.1:36786'
INFO - distributed.nanny -         Start Nanny at: 'tcp://127.0.0.1:33129'
INFO - dis

'http://127.0.0.1:8787/status'

In [9]:
%%time
# Preform checks
checks.validate_input(ds)

INFO - opera_tropo.checks - Performing checkup of input file
INFO - opera_tropo.checks -   Checking coordinate ranges and data variables.
INFO - opera_tropo.checks -   Checking nans and data valid range.
INFO - opera_tropo.checks -    Variable "q" stats:min = 0.00000, max = 0.02865, NaNs = 0
INFO - opera_tropo.checks -    Variable "z" stats:min = -4563.28223, max = 62700.71875, NaNs = 0
INFO - opera_tropo.checks -    Variable "t" stats:min = 182.19893, max = 317.13504, NaNs = 0
INFO - opera_tropo.checks -    Variable "lnsp" stats:min = 10.74795, max = 11.57485, NaNs = 0
CPU times: user 2.15 s, sys: 1.53 s, total: 3.67 s
Wall time: 20.2 s


<xarray.Dataset> Size: 29GB
Dimensions:    (time: 1, level: 137, latitude: 2560, longitude: 5120)
Coordinates:
  * longitude  (longitude) float32 20kB 2.501e-07 0.07031 0.1406 ... 359.9 359.9
  * latitude   (latitude) float32 10kB 89.95 89.88 89.81 ... -89.88 -89.95
  * level      (level) int32 548B 1 2 3 4 5 6 7 ... 131 132 133 134 135 136 137
  * time       (time) datetime64[ns] 8B 2024-09-10T06:00:00
Data variables:
    z          (time, level, latitude, longitude) float32 7GB dask.array<chunksize=(1, 137, 512, 1024), meta=np.ndarray>
    t          (time, level, latitude, longitude) float32 7GB dask.array<chunksize=(1, 137, 512, 1024), meta=np.ndarray>
    q          (time, level, latitude, longitude) float32 7GB dask.array<chunksize=(1, 137, 512, 1024), meta=np.ndarray>
    lnsp       (time, level, latitude, longitude) float32 7GB dask.array<chunksize=(1, 137, 512, 1024), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2025-05-21 02:32:18 GMT by grib_to_netcdf-2.31.0: grib_to_n...

In [10]:
%%time
ds = xr.open_dataset('/u/aurora-r0/govorcin/01_OPERA/TROPO/data/ECMWF_TROP_201606230600_201606230600_1.nc', chunks={})
vars_out = checks.check_nans_valid_range(ds)

WARNING - opera_tropo.checks -    Variable "q" is out of valid range (0.0, 0.3): min = -0.00001 [<0.0],max = 1.69043 [>0.3]
INFO - opera_tropo.checks -    Variable "z" stats:min = -4418.77490, max = 62685.22656, NaNs = 0
WARNING - opera_tropo.checks -    Variable "t" is out of valid range (140.0, 360.0): min = 82.57539 [<140.0],max = 606.28271 [>360.0]
INFO - opera_tropo.checks -    Variable "lnsp" stats:min = 10.74720, max = 11.57020, NaNs = 0
CPU times: user 2.58 s, sys: 1 s, total: 3.58 s
Wall time: 10.5 s


In [11]:
vars_out

{'q': [-1.3330401088751387e-05, 1.6904296875],
 't': [82.57538604736328, 606.28271484375]}

In [12]:
%%time
# Test it out
ds = checks.validate_input(ds)

INFO - opera_tropo.checks - Performing checkup of input file
INFO - opera_tropo.checks -   Checking coordinate ranges and data variables.


INFO - opera_tropo.checks -   Checking nans and data valid range.
WARNING - opera_tropo.checks -    Variable "q" is out of valid range (0.0, 0.3): min = -0.00001 [<0.0],max = 1.69043 [>0.3]
INFO - opera_tropo.checks -    Variable "z" stats:min = -4418.77490, max = 62685.22656, NaNs = 0
WARNING - opera_tropo.checks -    Variable "t" is out of valid range (140.0, 360.0): min = 82.57539 [<140.0],max = 606.28271 [>360.0]
INFO - opera_tropo.checks -    Variable "lnsp" stats:min = 10.74720, max = 11.57020, NaNs = 0
INFO - opera_tropo.checks - Clipping q below 0.0
INFO - opera_tropo.checks - Clipping q above 0.3
INFO - opera_tropo.checks - Clipping t below 140.0
INFO - opera_tropo.checks - Clipping t above 360.0
CPU times: user 2.59 s, sys: 986 ms, total: 3.58 s
Wall time: 10.7 s


In [13]:
%%time
# Test it out
ds = checks.validate_input(ds)

INFO - opera_tropo.checks - Performing checkup of input file
INFO - opera_tropo.checks -   Checking coordinate ranges and data variables.


INFO - opera_tropo.checks -   Checking nans and data valid range.
INFO - opera_tropo.checks -    Variable "q" stats:min = 0.00000, max = 0.29999, NaNs = 0
INFO - opera_tropo.checks -    Variable "z" stats:min = -4418.77490, max = 62685.22656, NaNs = 0
INFO - opera_tropo.checks -    Variable "t" stats:min = 140.00002, max = 360.00000, NaNs = 0
INFO - opera_tropo.checks -    Variable "lnsp" stats:min = 10.74720, max = 11.57020, NaNs = 0
CPU times: user 2.74 s, sys: 1.05 s, total: 3.79 s
Wall time: 11.3 s


In [16]:
client.close()

INFO - distributed.scheduler - Remove client Client-2065def2-6c14-11f0-a740-e43d1adad770
INFO - distributed.core - Received 'close-stream' from tcp://127.0.0.1:48622; closing.
INFO - distributed.scheduler - Remove client Client-2065def2-6c14-11f0-a740-e43d1adad770
INFO - distributed.scheduler - Close client connection: Client-2065def2-6c14-11f0-a740-e43d1adad770
INFO - distributed.scheduler - Retire worker addresses (stimulus_id='retire-workers-1753749473.8604555') (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15)
INFO - distributed.core - Received 'close-stream' from tcp://127.0.0.1:48600; closing.
INFO - distributed.core - Received 'close-stream' from tcp://127.0.0.1:48590; closing.
INFO - distributed.core - Received 'close-stream' from tcp://127.0.0.1:48618; closing.
INFO - distributed.core - Received 'close-stream' from tcp://127.0.0.1:48610; closing.
INFO - distributed.core - Received 'close-stream' from tcp://127.0.0.1:48606; closing.
INFO - distributed.core - Received 'clos

INFO - distributed.nanny - Closing Nanny at 'tcp://127.0.0.1:44627'. Reason: nanny-close
INFO - distributed.nanny - Nanny asking worker to close. Reason: nanny-close
INFO - distributed.nanny - Closing Nanny at 'tcp://127.0.0.1:35963'. Reason: nanny-close
INFO - distributed.nanny - Nanny asking worker to close. Reason: nanny-close
INFO - distributed.nanny - Closing Nanny at 'tcp://127.0.0.1:39225'. Reason: nanny-close
INFO - distributed.nanny - Nanny asking worker to close. Reason: nanny-close
INFO - distributed.nanny - Closing Nanny at 'tcp://127.0.0.1:39659'. Reason: nanny-close
INFO - distributed.nanny - Nanny asking worker to close. Reason: nanny-close
INFO - distributed.nanny - Closing Nanny at 'tcp://127.0.0.1:36786'. Reason: nanny-close
INFO - distributed.nanny - Nanny asking worker to close. Reason: nanny-close
INFO - distributed.nanny - Closing Nanny at 'tcp://127.0.0.1:33129'. Reason: nanny-close
INFO - distributed.nanny - Nanny asking worker to close. Reason: nanny-close
INFO